<a href="https://colab.research.google.com/github/DaraRahma536/TensorFlow-in-Action/blob/main/Chapter_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 11: Sequence-to-Sequence Learning**

# **1. Memahami Data Terjemahan Mesin**
---
Pada bagian ini, kita mempersiapkan dataset terjemahan Inggris-Jerman untuk model sequence-to-sequence (seq2seq). Dataset diambil dari http://www.manythings.org/anki/deu-eng.zip, yang berisi frasa paralel Inggris-Jerman.

Proses Persiapan Data:
* Memuat Data: Dataset dimuat ke dalam DataFrame pandas dengan kolom "EN" (Inggris) dan "DE" (Jerman).
* Pembersihan: Menghapus baris dengan karakter Unicode yang bermasalah.
* Penambahan Token Khusus: Menambahkan token sos (start of sentence) di awal dan eos (end of sentence) di akhir setiap terjemahan Jerman.
* Pembagian Data: Data dibagi menjadi 80% train, 10% validation, dan 10% test.
* Analisis Statistik: Menganalisis ukuran kosakata dan panjang sekuens untuk kedua bahasa.

Code Utama:

In [ ]:
# Memuat data
df = pd.read_csv('data/deu.txt', delimiter='\t', header=None)
df.columns = ["EN", "DE", "Attribution"]
df = df[["EN", "DE"]]

# Menambahkan token sos dan eos
df["DE"] = 'sos ' + df["DE"] + ' eos'

# Membagi data
train_df, valid_df, test_df = split_data(df)

# Analisis kosakata
en_vocab = get_vocabulary_size_greater_than(en_words, n=10)
de_vocab = get_vocabulary_size_greater_than(de_words, n=10)

# **2. Membangun Model Penerjemah Seq2Seq Inggris-Jerman**
---
Model seq2seq terdiri dari encoder dan decoder. Encoder memproses input bahasa sumber (Inggris) dan menghasilkan vektor konteks, yang kemudian digunakan decoder untuk menghasilkan terjemahan (Jerman).

Komponen Utama:
* TextVectorization Layer: Layer Keras yang mengubah string menjadi token IDs.
* Encoder: Menggunakan Bidirectional GRU untuk memahami konteks dari input Inggris.
* Decoder: Menggunakan GRU (non-bidirectional) dan dense layers untuk menghasilkan terjemahan kata per kata.

Code Pembuatan Model:

In [ ]:
# Mendefinisikan TextVectorization
en_vectorizer = TextVectorization(max_tokens=en_vocab+2, output_mode='int')
en_vectorizer.adapt(train_df["EN"])

# Mendefinisikan Encoder
encoder = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype=tf.string),
    en_vectorizer,
    tf.keras.layers.Embedding(en_vocab+2, 128, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128))
])

# Mendefinisikan Decoder
decoder = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype=tf.string),
    de_vectorizer,
    tf.keras.layers.Embedding(de_vocab+2, 128, mask_zero=True),
    tf.keras.layers.GRU(256, return_sequences=True),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(de_vocab+2, activation='softmax')
])

# Menggabungkan Encoder-Decoder
context_vector = encoder(english_input)
translation_output = decoder(german_input, initial_state=context_vector)

# **3. Melatih dan Mengevaluasi Model**
---
Model dilatih menggunakan teacher forcing, di mana decoder menerima input dari terjemahan sebenarnya (bukan prediksinya sendiri) selama pelatihan. Evaluasi menggunakan akurasi kata-per-kata dan skor BLEU.

Proses Pelatihan:
* Teacher Forcing: Decoder dilatih untuk memprediksi kata berikutnya berdasarkan kata sebelumnya yang sebenarnya.
* Custom Training Loop: Dilakukan karena memerlukan perhitungan BLEU yang tidak tersedia di Keras.
* Evaluasi: Dilakukan pada data validasi dan test menggunakan loss, akurasi, dan BLEU.

Code Pelatihan dan Evaluasi:

In [ ]:
# Fungsi evaluasi dengan BLEU
def evaluate_model(model, data, batch_size):
    bleu_scores = []
    for batch in data:
        pred = model.predict(batch)
        bleu = calculate_bleu(batch['target'], pred)
        bleu_scores.append(bleu)
    return np.mean(bleu_scores)

# Pelatihan dengan custom loop
for epoch in range(epochs):
    for batch in train_batches:
        model.train_on_batch(batch['input'], batch['target'])
    val_bleu = evaluate_model(model, val_data, batch_size)
    print(f"Epoch {epoch+1}, BLEU: {val_bleu}")

4. Dari Pelatihan ke Inferensi: Mendefinisikan Model Inferensi
Model yang dilatih dengan teacher forcing tidak dapat langsung digunakan untuk inferensi karena memerlukan input bahasa target (yang belum diketahui). Oleh karena itu, kita membuat model inferensi rekursif di mana decoder menggunakan prediksinya sendiri sebagai input untuk langkah berikutnya.

Perubahan pada Inferensi:
* Decoder Rekursif: Menggunakan output state GRU dari langkah sebelumnya sebagai initial state untuk langkah berikutnya.
* Stop Condition: Berhenti ketika decoder memprediksi token eos.

Code Model Inferensi:

In [ ]:
# Membuat model inferensi
def get_inference_model(trained_model):
    encoder = trained_model.get_layer("encoder")

    # Decoder untuk inferensi
    decoder_input = tf.keras.Input(shape=(1,), dtype=tf.string)
    state_input = tf.keras.Input(shape=(256,))

    # Menggunakan layer dari model yang sudah dilatih
    vectorized = de_vectorizer(decoder_input)
    embedded = embedding_layer(vectorized)
    gru_out = gru_layer(embedded, initial_state=state_input)
    final_out = dense_layer(gru_out)

    decoder = tf.keras.Model(
        inputs=[decoder_input, state_input],
        outputs=[final_out, gru_out]
    )
    return encoder, decoder

# Fungsi menghasilkan terjemahan
def translate(sentence, encoder, decoder):
    context_vector = encoder.predict([sentence])
    current_word = "sos"
    translation = []

    while current_word != "eos":
        pred, state = decoder.predict([[current_word], context_vector])
        current_word = de_vocabulary[np.argmax(pred)]
        translation.append(current_word)
        context_vector = state

    return " ".join(translation[:-1])  # Hapus 'eos'

# **Kesimpulan Utama**
---
* Model Seq2Seq cocok untuk tugas penerjemahan mesin karena dapat menangani input dan output dengan panjang variabel.
* Teacher Forcing mempercepat pelatihan dengan memberikan target sebenarnya sebagai input decoder.
* BLEU Score lebih baik daripada akurasi kata-per-kata untuk mengevaluasi kualitas terjemahan.
* Model Inferensi memerlukan modifikasi khusus karena decoder harus bekerja secara rekursif menggunakan prediksinya sendiri.